### Physical activity attenuates the associations of systemic immune-inflammation index with total and cause-specific mortality among middle-aged and older populations
https://www.nature.com/articles/s41598-021-91324-x#Sec6

#### Results
Subjects in the high SII subgroup had increased risks of total, CVD, cancer, and other causes mortality 
[HR (95%CI) = 1.38 (1.27–1.49), 1.33 (1.11–1.59), 1.22 (1.04–1.45) and 1.47 (1.32–1.63), respectively]

#### Study populations
In the NHANES 1999–2014, after excluding individuals who did not take health examination (n = 1487), without mortality data (n = 38), and without information of related blood cells count (n = 1318), a total of 25,761 participants aged ≥ 40 years old were included because most deaths occurred in these adults

...
...

In [ ]:
pip install google-cloud-bigquery

In [8]:
pip install db-dtypes

  Obtaining dependency information for db-dtypes from https://files.pythonhosted.org/packages/4e/5c/72b74daf6655db6f6afc2b3bc44359c7953004ef4630325a3ae0efc50872/db_dtypes-1.2.0-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [144]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "nhanesgcp-93d0f7365fdc.json"

In [145]:
from google.cloud import bigquery
client = bigquery.Client()

In [650]:
query = """
WITH SmokingData AS (
    SELECT
        respondent_sequence_number,
        smoked_at_least_100_cigarettes_in_life
    FROM
        `nhanesgcp.dbt.smoking_cigarette_use_questionnaire`
),
AlcoholData AS (
    SELECT
        respondent_sequence_number,
        had_at_least_12_alcohol_drinks_1_yr
    FROM
        `nhanesgcp.dbt.alcohol_use_questionnaire`
),
SmokingStatus AS (
    SELECT
        respondent_sequence_number,
        CASE
            WHEN smoked_at_least_100_cigarettes_in_life IS NULL THEN NULL
            WHEN smoked_at_least_100_cigarettes_in_life = 'Yes' THEN 'Smoker'
            ELSE 'Non-Smoker'
        END AS smoking_status
    FROM
        SmokingData
),
AlcoholDrinkingStatus AS (
    SELECT
        respondent_sequence_number,
        CASE
            WHEN had_at_least_12_alcohol_drinks_1_yr IS NULL THEN NULL
            WHEN had_at_least_12_alcohol_drinks_1_yr = 'Yes' THEN 'Drinker'
            ELSE 'Non-Drinker'
        END AS alcohol_drinking_status
    FROM
        AlcoholData
),
PhysicalActivityData AS (
    SELECT
        respondent_sequence_number,
        COALESCE(SUM(CASE
                WHEN vigorous_activity_over_past_30_days = 'Yes' THEN of_times_past_30_days * how_long_each_time_minutes
                ELSE 0
            END), 0) +
        COALESCE(SUM(CASE
                WHEN moderate_activity_over_past_30_days = 'Yes' THEN of_times_past_30_days * how_long_each_time_minutes
                ELSE 0
            END), 0) AS total_activity_duration_minutes
    FROM
        `nhanesgcp.dbt.physical_activity_questionnaire`
    GROUP BY
        respondent_sequence_number
),
SII_Calculation AS (
    SELECT 
        respondent_sequence_number,
        (segmented_neutrophils_num_1000_cell_ul * platelet_count_1000_cells_ul) / lymphocyte_number_1000_cells_ul AS SII
    FROM 
        `nhanesgcp.dbt.complete_blood_count_with_5_part_differential_whole_blood_laboratory`
),
Filtered_Demographics AS (
    SELECT 
        respondent_sequence_number,
        age_in_years_at_screening AS age,
        gender,
        race_hispanic_origin AS race
    FROM 
        `nhanesgcp.dbt.demographic_variables_sample_weights_demographics`
    WHERE 
        end_year <= 2015
        AND 
        CAST(age_in_years_at_screening AS INT64) >= 40
),
BodyMeasurements AS (
    SELECT 
        respondent_sequence_number,
        weight_kg,
        standing_height_cm,
        weight_kg / (standing_height_cm / 100) AS bmi
    FROM 
        `nhanesgcp.dbt.body_measures_examination`
),
PhysicalActivityStatus AS (
    SELECT
        respondent_sequence_number,
        CASE
            WHEN COALESCE(total_activity_duration_minutes, 0) >= 150 THEN 'Physically Active'
            ELSE 'Physically Inactive'
        END AS physical_activity_status
    FROM
        PhysicalActivityData
),
MortalityData AS (
    SELECT
        respondent_sequence_number,
        ucod_leading,
        mort_stat,
        permth_exm
    FROM
        `nhanesgcp.nhanes_mort_1999_2018.nhanes_mort`
),
SII_Stats AS (
    SELECT
        APPROX_QUANTILES(SII, 4) AS quartiles
    FROM
        SII_Calculation
),
SII_Groups AS (
    SELECT
        SII_Calculation.*,
        CASE
            WHEN SII IS NULL THEN NULL
            WHEN SII <= (SELECT quartiles[OFFSET(1)] FROM SII_Stats) THEN 'Low'
            WHEN SII <= (SELECT quartiles[OFFSET(2)] FROM SII_Stats) THEN 'Middle'
            ELSE 'High'
        END AS SII_Group
    FROM
        SII_Calculation
    CROSS JOIN
        SII_Stats
)
SELECT 
    fd.respondent_sequence_number,
    fd.age,
    fd.gender,
    fd.race,
    bm.bmi,
    sii.SII,
    ss.smoking_status,
    ads.alcohol_drinking_status,
    pa.total_activity_duration_minutes AS physical_activity_duration_minutes,
    pas.physical_activity_status,
    md.ucod_leading,
    md.mort_stat,
    md.permth_exm,
    sg.SII_Group
FROM 
    Filtered_Demographics fd
LEFT JOIN 
    BodyMeasurements bm ON fd.respondent_sequence_number = bm.respondent_sequence_number
LEFT JOIN 
    SII_Calculation sii ON fd.respondent_sequence_number = sii.respondent_sequence_number
LEFT JOIN 
    SmokingStatus ss ON fd.respondent_sequence_number = ss.respondent_sequence_number
LEFT JOIN 
    AlcoholDrinkingStatus ads ON fd.respondent_sequence_number = ads.respondent_sequence_number
LEFT JOIN 
    SmokingData smoking ON fd.respondent_sequence_number = smoking.respondent_sequence_number
LEFT JOIN 
    AlcoholData alcohol ON fd.respondent_sequence_number = alcohol.respondent_sequence_number
LEFT JOIN
    PhysicalActivityData pa ON fd.respondent_sequence_number = pa.respondent_sequence_number
LEFT JOIN
    PhysicalActivityStatus pas ON fd.respondent_sequence_number = pas.respondent_sequence_number
LEFT JOIN
    MortalityData md ON fd.respondent_sequence_number = md.respondent_sequence_number
LEFT JOIN
    SII_Groups sg ON fd.respondent_sequence_number = sg.respondent_sequence_number
WHERE
    sii.SII IS NOT NULL
ORDER BY
    fd.respondent_sequence_number;

"""

In [651]:
df = client.query(query).to_dataframe()

In [652]:
df.head(50)

,respondent_sequence_number,age,gender,race,bmi,SII,smoking_status,alcohol_drinking_status,physical_activity_duration_minutes,physical_activity_status,ucod_leading,mort_stat,permth_exm,SII_Group
0,2,77.0,Male,Non-Hispanic White,43.333333,682.125000,None,None,0.0,Physically Inactive,6.0,1,177,High
1,5,49.0,Male,Non-Hispanic White,51.878856,294.500000,None,None,2040.0,Physically Active,NaN,0,244,Low
2,7,59.0,Female,Non-Hispanic Black,47.882136,222.782609,None,None,12.0,Physically Inactive,NaN,0,236,Low
3,10,43.0,Male,Non-Hispanic Black,58.811152,175.789474,None,None,0.0,Physically Inactive,1.0,1,231,Low
4,13,70.0,Male,Mexican American,40.329740,1459.200000,None,None,0.0,Physically Inactive,1.0,1,16,High
5,14,81.0,Male,Non-Hispanic White,45.427196,337.777778,None,None,1080.0,Physically Active,3.0,1,136,Middle
6,16,85.0,Female,Non-Hispanic Black,28.779473,500.571429,None,None,780.0,Physically Active,2.0,1,62,High
7,24,53.0,Female,Non-Hispanic White,42.570037,438.351351,None,None,0.0,Physically Inactive,2.0,1,160,Middle
8,25,42.0,Female,Non-Hispanic White,62.687463,531.423077,None,None,0.0,Physically Inactive,10.0,1,142,High
9,29,62.0,Male,Non-Hispanic White,64.608348,4930.000000,None,None,0.0,Physically Inactive,3.0,1,26,High


In [653]:
df

,respondent_sequence_number,age,gender,race,bmi,SII,smoking_status,alcohol_drinking_status,physical_activity_duration_minutes,physical_activity_status,ucod_leading,mort_stat,permth_exm,SII_Group
0,2,77.0,Male,Non-Hispanic White,43.333333,682.125000,None,None,0.0,Physically Inactive,6.0,1,177,High
1,5,49.0,Male,Non-Hispanic White,51.878856,294.500000,None,None,2040.0,Physically Active,NaN,0,244,Low
2,7,59.0,Female,Non-Hispanic Black,47.882136,222.782609,None,None,12.0,Physically Inactive,NaN,0,236,Low
3,10,43.0,Male,Non-Hispanic Black,58.811152,175.789474,None,None,0.0,Physically Inactive,1.0,1,231,Low
4,13,70.0,Male,Mexican American,40.329740,1459.200000,None,None,0.0,Physically Inactive,1.0,1,16,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22220,71902,67.0,Female,Other Race - Including Multi-Racial,58.743169,733.700000,Non-Smoker,Drinker,0.0,Physically Inactive,1.0,1,94,High
22221,71907,80.0,Male,Non-Hispanic White,40.763968,291.214286,Non-Smoker,Drinker,0.0,Physically Inactive,4.0,1,84,Low
22222,71908,66.0,Female,Non-Hispanic White,55.786164,352.444444,Non-Smoker,Non-Drinker,0.0,Physically Inactive,NaN,0,103,Middle
22223,71912,40.0,Male,Non-Hispanic White,49.520587,617.615385,Smoker,Drinker,0.0,Physically Inactive,NaN,0,95,High


In [654]:
cph = CoxPHFitter()


# columns_to_exclude = ['age','gender','race','bmi','smoked_at_least_100_cigarettes_in_life','had_at_least_12_alcohol_drinks_1_yr','smoking_status','alcohol_drinking_status','physical_activity_duration_minutes']
# df = df.drop(columns=columns_to_exclude)

# df_cleaned = df.dropna()
df_cleaned['SII_Group'] = pd.Categorical(df_cleaned['SII_Group'], categories=['Low', 'Middle', 'High'], ordered=True)

cph.fit(df_cleaned, duration_col='permth_exm', event_col='mort_stat', formula="SII_Group")

<lifelines.CoxPHFitter: fitted with 2643 total observations, 0 right-censored observations>

In [655]:
 cph.summary

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p)
covariate,,,,,,,,,,,
SII_Group[T.Middle],-0.093138,0.911068,0.064772,-0.220089,0.033814,0.802447,1.034392,0.0,-1.437924,0.150456,2.732590
SII_Group[T.High],-0.056702,0.944876,0.053151,-0.160876,0.047472,0.851397,1.048617,0.0,-1.066808,0.286059,1.805617


In [641]:

# df_cleaned = df_cleaned.dropna(subset=['ucod_leading'])
cph.fit(df_cleaned, duration_col='permth_exm', event_col='mort_stat',  formula="SII_Group")

hazard_ratios = cph.summary['exp(coef)']
print("Hazard Ratios:")
print(hazard_ratios)

cph.summary

Hazard Ratios:
covariate
SII_Group[T.Middle]    0.911068
SII_Group[T.High]      0.944876
Name: exp(coef), dtype: float64


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p)
covariate,,,,,,,,,,,
SII_Group[T.Middle],-0.093138,0.911068,0.064772,-0.220089,0.033814,0.802447,1.034392,0.0,-1.437924,0.150456,2.732590
SII_Group[T.High],-0.056702,0.944876,0.053151,-0.160876,0.047472,0.851397,1.048617,0.0,-1.066808,0.286059,1.805617


In [642]:
cause_specific_df = df_cleaned[df_cleaned['ucod_leading'] == 1]
cph.fit(cause_specific_df, duration_col='permth_exm', event_col='mort_stat', formula="SII_Group")
hazard_ratios = cph.summary['exp(coef)']
print("Hazard Ratios for Cause-Specific Mortality (Other causes):")
print(hazard_ratios)
cph.summary

Hazard Ratios for Cause-Specific Mortality (Other causes):
covariate
SII_Group[T.Middle]    1.024043
SII_Group[T.High]      1.067418
Name: exp(coef), dtype: float64


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p)
covariate,,,,,,,,,,,
SII_Group[T.Middle],0.023758,1.024043,0.138296,-0.247298,0.294814,0.780908,1.342876,0.0,0.171791,0.863602,0.211562
SII_Group[T.High],0.065243,1.067418,0.113292,-0.156806,0.287292,0.854870,1.332813,0.0,0.575883,0.564694,0.824458
